# Nearest Neighbour Accuracy

1. load the model
2. function that assesses the accuracy of a nearest neighbor algorithm in predicting targets for one-shot tasks.



In [1]:
import tensorflow
from keras.optimizers import Adam
import time
import os
import numpy as np
from model_architecture import get_model

c:\Users\satha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 4096)      │ 38,960,448 │ input_layer[0][0… │
│ (Sequential)        │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 4096)      │          0 │ sequential[0][0], │
│                     │                   │            │ sequential[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │      4,097 │ lambda[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,964,545 (148.64 MB)

 Trainable params: 38,964,545 (148.64 MB)

 Non-trainable params: 0 (0.00 B)

In [2]:
# Produce inputs and their corresponding targets
# Aim : To produce a data genertor for our model
import tensorflow as tf
from keras.optimizers import Adam
import numpy as np
import pickle
import os
from sklearn.utils import shuffle

# #_______________________________________________________________________
save_path = '../Data/'

with open(os.path.join(save_path, "train_data.pickle"), 'rb') as f:
    (train_data, train_range) = pickle.load(f)

with open(os.path.join(save_path, "val_data.pickle"), 'rb') as f:
    (val_data, val_range) = pickle.load(f)

rng = np.random.RandomState(seed=42)
# ____________________________________________________________________


def get_batch(batch_size, s="train"):  # defualt training set

    image_data = train_data if s == "train" else val_data

    if len(image_data.shape) == 5:
        (no_of_classes, no_of_images, height, width, scale) = image_data.shape
    else:
        (no_of_classes, no_of_images, height, width) = image_data.shape
        scale = 1 

    class_categories = rng.choice(
        no_of_classes, size=(batch_size,), replace=False)

    # initialize container to store pair of images and their corresponding targets
    empty_image = np.zeros((batch_size, height, width, scale))
    pairs = [empty_image for i in range(2)]
    targets = np.zeros((batch_size,))

    # set 2nd half of targets to 1
    targets[(batch_size // 2):] = 1

    # insert pairs logic
    for i in range(batch_size):
        category_1 = class_categories[i]

        # each category has 20 images so we select a random one
        index_1 = rng.randint(0, no_of_images)
        index_2 = rng.randint(0, no_of_images)
        # fill 1st image i.e. pairs[0]

        pairs[0][i, :, :, :] = image_data[category_1,
                                          index_1].reshape(width, height, scale)
        # now we fill 2nd image according to our target label
        if i >= (batch_size//2):
            category_2 = category_1  # same characters(may be different images)
        else:
            # category is an integer less than or equal to no_of_classes,
            # hence we add any random integer(not equal to zero) to get new category
            # which will represent new class
            category_2 = (category_1 + rng.randint(1,
                          no_of_classes)) % no_of_classes
            # % ensures that the new value is with in the bound

        # fill 2nd image
        pairs[1][i, :, :, :] = image_data[category_2,
                                          index_2].reshape(width, height, scale)
    return pairs, targets

# ________________________________________________________________________________________________-


def data_generator(batch_size, s="train"):
    while True:
        pairs, targets = get_batch(batch_size, s)
        yield pairs, targets


# Test Data Generation
# 1. function to generate pairs of test images and support set.
# 2. we fill array for test images to N copies of one class (N user defined)
# 3. support set will contain N classes, which will include the above class


def make_oneshot_task(N, s="val", language=None):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    if s == 'train':
        X = train_data
        categories = train_range
    else:
        X = val_data
        categories = val_range

    if len(X.shape) == 5:
        (n_classes, n_examples, w, h, s) = X.shape
    else:
        (n_classes, n_examples, w, h) = X.shape
        s = 1 

    indices = rng.randint(0, n_examples, size=(N,))
    if language is not None:  # if language is specified, select characters for that language
        low, high = categories[language]
        if N > high - low:
            raise ValueError(
                "This language ({}) has less than {} letters".format(language, N))
        categories = rng.choice(range(low, high), size=(N,), replace=False)

    else:  # if no language specified just pick a bunch of random letters
        categories = rng.choice(range(n_classes), size=(N,), replace=False)
    true_category = categories[0]
    ex1, ex2 = rng.choice(n_examples, replace=False, size=(2,))
    test_image = np.asarray(
        [X[true_category, ex1, :, :]]*N).reshape(N, w, h,s)
    support_set = X[categories, indices, :, :]
    support_set[0, :, :] = X[true_category, ex2]
    support_set = support_set.reshape(N, w, h,s)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(
        targets, test_image, support_set)
    pairs = [test_image, support_set]

    return pairs, targets

# ________________________________________________________________)_______________________________

# #### Testing the model
# 1. now we use above function to get test data
# 2. predict this data with the model
# 3. print relevant information


def test_oneshot(model, N, k, s="val", verbose=0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k, N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N, s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct += 1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct, N))
    return percent_correct
######################################################################################################################


In [3]:

def get_iteration_from_filename(filename):
    return int(filename.split('.')[1])

def get_max_iteration_and_model(model_path):
    # Get list of all weight files
    weight_files = os.listdir(model_path)

    # Filter out non-weight files and extract iteration numbers
    iteration_weights = [(get_iteration_from_filename(file), file) for file in weight_files if file.endswith('.h5')]

    if iteration_weights:
        # Select the weight file with maximum iteration number
        max_iteration_file = max(iteration_weights, key=lambda x: x[0])[1]
        max_iteration = iteration_weights[-1][0]  # Get maximum iteration number
        model = get_model()
        model.load_weights(os.path.join(model_path, max_iteration_file))
        return max_iteration, model
    else:
        return 0, None  # No weight files found

model_path = './weights/'
max_iteration, model = get_max_iteration_and_model(model_path)

if model is not None:
    print("Loaded model with maximum iteration:", max_iteration)
    # Additional code using the loaded model
else:
    print("No weight files found in the specified path.")

c:\Users\satha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Loaded model with maximum iteration: 7300


In [4]:
def nearest_neighbour_correct(pairs, targets):
    """returns 1 if nearest neighbour gets the correct answer for a one-shot task
       given by (pairs, targets)"""
    L2_distances = np.zeros(len(targets))  
    for i in range(len(targets)):
        L2_distances[i] = np.sum(np.sqrt(pairs[0][i]**2 - pairs[1][i]**2)) 
    if np.argmin(L2_distances) == np.argmax(targets):
        return 1
    return 0


In [5]:
def test_nn_accuracy(N_ways,n_trials):
    """Returns accuracy of nearest neighbour approach """
    print("Evaluating nearest neighbour on {} unique {} way one-shot learning tasks ...".format(n_trials,N_ways))

    n_right = 0
    
    for i in range(n_trials):
        pairs,targets = make_oneshot_task(N_ways,"val")
        correct = nearest_neighbour_correct(pairs,targets)
        n_right += correct
    return 100.0 * n_right / n_trials

In [6]:
ways = np.arange(1,20,2) # contains 1,3,5,7...
resume =  False
trials = 50

In [7]:
val_accs, train_accs,nn_accs = [], [], []
for N in ways:    
    val_accs.append(test_oneshot(model, N, trials, "val"))
    train_accs.append(test_oneshot(model, N, trials, "train"))
    nn_acc = test_nn_accuracy(N, trials)
    nn_accs.append(nn_acc)
    print ("NN Accuracy = ", nn_acc)
    print("---------------------------------------------------------------------------------------------------------------")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━

c:\Users\satha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step

In [8]:
import pickle
save_path = '../Data/'

with open(os.path.join(save_path,"accuracies.pickle"), "wb") as f:
    pickle.dump((val_accs,train_accs,nn_accs),f)    